In [30]:
import pandas as pd

df = pd.read_csv("best_sellers_data2.csv")
print(df.head())



                                       product_title product_price  \
0  TurboTax Deluxe 2024 Tax Software, Federal & S...        $55.99   
1  TurboTax Premier 2024 Tax Software, Federal & ...        $82.99   
2  TurboTax Home & Business 2024 Tax Software, Fe...        $95.99   
3  TurboTax Business 2024 Tax Software, Federal T...       $143.99   
4  H&R Block Tax Software Deluxe + State 2024 wit...        $49.97   

   product_star_rating  product_num_ratings  ranks country  
0                  4.2               6511.0      1      US  
1                  4.1               2738.0      2      US  
2                  4.2               1672.0      3      US  
3                  4.0                389.0      4      US  
4                  3.9               1683.0      5      US  


In [31]:
import pandas as pd
import os

file_path = "best_sellers_data2.csv"  

try:
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File '{file_path}' does not exist.")

    df = pd.read_csv(file_path)

    
    print("File loaded successfully.")
    # print(df.head())

except FileNotFoundError as fnf_err:
    print("Error:", fnf_err)

except pd.errors.EmptyDataError:
    print("Error: The file is empty.")

except pd.errors.ParserError:
    print("Error: The file could not be parsed correctly (maybe malformed CSV).")

except Exception as e:
    print("An unexpected error occurred:", str(e))

File loaded successfully.


In [34]:
import pandas as pd
import mysql.connector


df = pd.read_csv("best_sellers_data2.csv")


df['product_price'] = df['product_price'].replace(r'[\$,€\xa0,]', '', regex=True)
df['product_price'] = pd.to_numeric(df['product_price'], errors='coerce')


df['product_star_rating'] = pd.to_numeric(df['product_star_rating'], errors='coerce')
df['product_num_ratings'] = pd.to_numeric(df['product_num_ratings'], errors='coerce')


df.dropna(subset=['product_title', 'product_price', 'product_star_rating', 'product_num_ratings'], inplace=True)


df['ranks'] = df['ranks'].fillna('Unknown')
df['country'] = df['country'].fillna('Unknown')
print("Cleaned Data Shape:", df.shape)
print(df.head())


conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Samyuktha123',
    database='amazon'
)

cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS software_products (
        product_id INT AUTO_INCREMENT PRIMARY KEY,
        product_title TEXT,
        product_price FLOAT,
        product_star_rating FLOAT,
        product_num_ratings INT,
        ranks TEXT,
        country VARCHAR(100)
    )
""")
conn.commit()


for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO software_products (
            product_title,
            product_price,
            product_star_rating,
            product_num_ratings,
            ranks,
            country
        )
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['product_title'],
        row['product_price'],
        row['product_star_rating'],
        row['product_num_ratings'],
        row['ranks'],
        row['country']
    ))
    
cursor.close()
conn.close()

print("✅ Data inserted into MySQL successfully.")

Cleaned Data Shape: (1353, 6)
                                       product_title  product_price  \
0  TurboTax Deluxe 2024 Tax Software, Federal & S...          55.99   
1  TurboTax Premier 2024 Tax Software, Federal & ...          82.99   
2  TurboTax Home & Business 2024 Tax Software, Fe...          95.99   
3  TurboTax Business 2024 Tax Software, Federal T...         143.99   
4  H&R Block Tax Software Deluxe + State 2024 wit...          49.97   

   product_star_rating  product_num_ratings  ranks country  
0                  4.2               6511.0      1      US  
1                  4.1               2738.0      2      US  
2                  4.2               1672.0      3      US  
3                  4.0                389.0      4      US  
4                  3.9               1683.0      5      US  
✅ Data inserted into MySQL successfully.


In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv("best_sellers_data2.csv")


df['product_price'] = df['product_price'].replace(r'[\$,€\xa0,]', '', regex=True)
df['product_price'] = pd.to_numeric(df['product_price'], errors='coerce')


df['product_star_rating'] = pd.to_numeric(df['product_star_rating'], errors='coerce')


df['product_num_ratings'] = pd.to_numeric(df['product_num_ratings'], errors='coerce')


df.dropna(subset=['product_title', 'product_price', 'product_star_rating', 'product_num_ratings'], inplace=True)


top_rated = df.sort_values(by='product_star_rating', ascending=False).head(10)
print("Top 10 Best-Selling Software by Rating:")
print(top_rated[['product_title', 'product_star_rating']])


if 'category' in df.columns:
    avg_price_by_category = df.groupby('category')['product_price'].mean().sort_values(ascending=False)
    print("\nAverage Price of Software Per Category:")
    print(avg_price_by_category)


correlation = df[['product_star_rating', 'product_num_ratings']].corr()
print("\nCorrelation between Rating and Review Count:")
print(correlation)

Q1_price = df['product_price'].quantile(0.25)
Q3_price = df['product_price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

price_outliers = df[(df['product_price'] < lower_bound_price) | (df['product_price'] > upper_bound_price)]
print("\nOutliers in Product Price:")
print(price_outliers[['product_title', 'product_price']])


Q1_rating = df['product_star_rating'].quantile(0.25)
Q3_rating = df['product_star_rating'].quantile(0.75)
IQR_rating = Q3_rating - Q1_rating
lower_bound_rating = Q1_rating - 1.5 * IQR_rating
upper_bound_rating = Q3_rating + 1.5 * IQR_rating

rating_outliers = df[(df['product_star_rating'] < lower_bound_rating) | (df['product_star_rating'] > upper_bound_rating)]
print("\nOutliers in Product Ratings:")
print(rating_outliers[['product_title', 'product_star_rating']])

Top 10 Best-Selling Software by Rating:
                                          product_title  product_star_rating
792                               Microsoft 365 Empresa                  5.0
2329  Norton 360 Deluxe 2024, antivirussoftware, int...                  5.0
251             Microsoft Office Home and Business 2024                  5.0
1641  McAfee®+ Premium Individual Antivirus & Intern...                  5.0
1642  McAfee® Total Protection Antivirus & Internet ...                  5.0
2333  Norton 360 Premium 2024, antivirussoftware, in...                  5.0
2336  Norton 360 Premium 2024, antivirussoftware, in...                  5.0
231   Avast Ultimate 5-User 1Yr with Secureline VPN ...                  5.0
1673  Microsoft Windows Server 2022 Standard 64-bit ...                  5.0
215   TurboImpôt Premier 2024 - 12 déclarations - Fr...                  5.0

Correlation between Rating and Review Count:
                     product_star_rating  product_num_ratings
produ